#### A

In [6]:
# Set Hyper Parameters
NAME = 'model' #모델의 이름을 입력합니다 예시: 'model'

<br></br>

#### B

In [ ]:
import jajucha2  # jajucha2 라이브러리를 가져옵니다
import numpy as np  # 넘파이 라이브러리, 배열 및 수치 계산에 사용
import cv2  # OpenCV 라이브러리, 컴퓨터 비전 및 이미지 처리에 사용
import torch  # PyTorch 라이브러리를 가져옵니다 (AI 모델 결과를 처리하기 위함)

# 초기 조향각과 속도를 0으로 설정
steer = 0
speed = 0

# 무한 루프 시작 (계속해서 주행 로직이 실행됨)
while True:
    # 카메라로부터 중앙 이미지 캡처
    image = jajucha2.camera.get_image('center')

    # 그리드 기반 주행 코드: 이미지에서 그리드를 분석하여 좌, 우, 전방 상태를 확인
    (V, L, R), grid = jajucha2.camera.gridFront(image)

    # 전방 그리드의 특정 인덱스 값이 100보다 작으면 왼쪽으로 조향(-10)하고, 속도는 5로 설정
    if(V[5] < 100):
        steer = -10
        speed = 5
    else:
        steer = 0  # 그렇지 않으면 조향을 0으로 유지
        speed = 5  # 속도는 그대로 5

    # 깊이 기반 거리 감지 주행
    depth = jajucha2.camera.get_depth()  # 깊이 정보를 가져옴
    jajucha2.camera.show_image(depth, 'depth')  # 깊이 이미지를 화면에 표시
    height, width = depth.shape[:2]  # 이미지의 높이와 너비를 가져옴
    center_x, center_y = width // 2, height // 2  # 중앙 좌표 계산
    region_size = 60  # 중심 영역의 크기 설정
    start_x = center_x - region_size // 2  # 중심 영역의 시작 x좌표
    start_y = center_y - region_size // 2  # 중심 영역의 시작 y좌표
    center_region = depth[start_y:start_y + region_size, start_x:start_x + region_size]  # 중심 영역 추출
    mean_value = np.mean(center_region)  # 중심 영역의 평균 거리 값 계산
    if(mean_value > 100):  # 평균 거리 값이 100보다 크면 속도를 0으로 설정 (장애물 감지)
        speed = 0

    ## 인공지능 기반 신호등 감지
    output = jajucha2.ai.get_model_output(image, NAME)  # 이미지에서 AI 모델의 출력 결과를 가져옴
    max_index = torch.argmax(output)  # AI 모델 출력에서 가장 높은 값을 가진 인덱스를 가져옴
    if(max_index == 1):  # 인덱스 1이 감지되면 (예: 빨간 신호등) 속도를 0으로 설정
        speed = 0

    # 조향 및 속도를 설정하여 모터 제어
    jajucha2.control.set_motor(steer, steer, speed)
